In [1]:
!pip install openai -U

import json
import threading
import itertools
import time
from datetime import datetime
from openai import OpenAI

# Your OpenAI API key
# bot1 = OpenAI(
#     api_key=""
# )

# Initialize conversation history
customer_service_bot_system_prompt = {
    "role": "system", "content":
    """
    You are a helpful customer service bot that will help the customer with product information, pricing, and availability.
    
    But when the customer asks or mentions about product suggestions, 
     
    I want you to reply with "For more helpful product suggestions I will transfer you to Sales Bot"
    """
}

sales_bot_system_prompt = {
    "role": "system", "content":
    """
    You are a helpful sales bot that will help the customer with product suggestions
    """
}

conversation_history = []

# Spinner function that runs in a separate thread
def spinner():
    for char in itertools.cycle('|/-\\'):
        if not spin_flag:
            break
        print(f'\rProcessing {char}', end='', flush=True)
        time.sleep(0.1)
    print('\r', end='', flush=True)  # Clear the line when done

# Function to get a response from the ChatGPT API
def chatbot(messages):
    response = bot1.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=150
    )
    return response.choices[0].message.content


def get_dynamic_file_name():
    now = datetime.utcnow()  # Get current time in GMT
    return now.strftime("conversation_%Y-%m-%d_%H-%M-%S.json")

# Function to save conversation history to a JSON file with a dynamic name
def save_conversation_to_json():
    file_name = get_dynamic_file_name()
    with open(file_name, 'w') as file:
        json.dump(conversation_history, file, indent=4)

# Main chat loop
def chat(customer_service_or_sales: bool):
    if customer_service_or_sales:
        bot_system_prompt = customer_service_bot_system_prompt
        bot_type = "Customer Service Bot"
    else:
        bot_system_prompt = sales_bot_system_prompt
        bot_type = "Sales Bot"

    # Set the appropriate system prompt in the conversation history
    if conversation_history:
        conversation_history[0] = bot_system_prompt
    else:
        conversation_history.append(bot_system_prompt)

    greetings = f"Hi! How can I help you today?"
    print(f"{bot_type}: {greetings}")
    conversation_history.append({"role": "assistant", "content": greetings})


    while True:
        user_input = input("You: ")

        # Add user's message to conversation history
        conversation_history.append({"role": "user", "content": user_input})
        print(f"You: {user_input}")

        # Start the spinner in a separate thread
        global spin_flag
        spin_flag = True
        spinner_thread = threading.Thread(target=spinner)
        spinner_thread.start()

        # Get response from ChatGPT
        response = chatbot(conversation_history)
        
        # Stop the spinner once the response is ready
        spin_flag = False
        spinner_thread.join()

        # Add the chatbot's response to conversation history
        conversation_history.append({"role": "assistant", "content": response})

        print(f"{bot_type}: {response}")

        if user_input.lower() in ['exit', 'quit', 'bye']:
            save_conversation_to_json()
            print("Chatbot: Goodbye!")
            break
        elif "transfer you to Sales Bot" in response:
            print("Transferring to Sales Bot...")
            save_conversation_to_json()
            chat(customer_service_or_sales=False)
            break
        elif "transfer you to Customer Service Bot" in response:
            print("Transferring to Customer Service Bot...")
            save_conversation_to_json()
            chat(customer_service_or_sales=True)
            break

chat(customer_service_or_sales=True)

Customer Service Bot: Hi! How can I help you today?
You: Is Jordan 1 Pine Green Available in your online store?
Customer Service Bot: I’ll need to check the latest availability for you. The Jordan 1 Pine Green is a popular model, so it may vary. Please give me a moment. 

As of now, I don't have the current stock information, but you can find it on our online store. If it’s out of stock, you might be able to sign up for restock notifications. Let me know if you need more information!
You: What is the price of this shoes?
Customer Service Bot: The price of the Jordan 1 Pine Green can vary based on factors like size and retailer. Typically, it ranges from around $160 to $220. However, prices may fluctuate due to demand and availability. To get the most accurate pricing, I recommend checking our online store or contacting customer service directly. If you have any other questions, feel free to ask!
You: Does this shoe comes with a special box?
Customer Service Bot: Yes, the Jordan 1 Pine 